In [1]:
import mitsuba as mi
mi.set_variant('llvm_ad_rgb')

In [2]:
scene = mi.load_file('scenes/cbox/cbox.xml')
image_ref = mi.render(scene, spp=64)
mi.util.convert_to_bitmap(image_ref)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]

In [3]:
scene_params = mi.traverse(scene)
type(scene_params)

mitsuba.python.util.SceneParameters

In [6]:
scene_params

SceneParameters[
  ------------------------------------------------------------------------------------------
  Name                                   Flags    Type            Parent
  ------------------------------------------------------------------------------------------
  PerspectiveCamera.near_clip                     float           PerspectiveCamera
  PerspectiveCamera.far_clip                      float           PerspectiveCamera
  PerspectiveCamera.shutter_open                  float           PerspectiveCamera
  PerspectiveCamera.shutter_open_time             float           PerspectiveCamera
  PerspectiveCamera.film.size                     ScalarVector2u  HDRFilm
  PerspectiveCamera.film.crop_size                ScalarVector2u  HDRFilm
  PerspectiveCamera.film.crop_offset              ScalarPoint2u   HDRFilm
  PerspectiveCamera.x_fov                         Float           PerspectiveCamera
  PerspectiveCamera.to_world                      Transform4f     PerspectiveCamer

In [7]:
param_key = 'red.reflectance.value'
param_ref = mi.Color3f(scene_params[param_key])
scene_params[param_key] = mi.Color3f(0.01, 0.0, 0.9)

In [8]:
scene_params.update();

In [10]:
image = mi.render(scene, spp=1024)
mi.util.convert_to_bitmap(image)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]

### Goal: Recover original scene, by optimizing BSDF reflectance value

In [11]:
opt = mi.ad.Adam(lr=0.05)

In [12]:
help(mi.ad.Adam)

Help on class Adam in module mitsuba.ad.optimizers:

class Adam(Optimizer)
 |  Adam(lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, mask_updates=False, uniform=False, params: dict = None)
 |  
 |  Implements the Adam optimizer presented in the paper *Adam: A Method for
 |  Stochastic Optimization* by Kingman and Ba, ICLR 2015.
 |  
 |  When optimizing many variables (e.g. a high resolution texture) with
 |  momentum enabled, it may be beneficial to restrict state and variable
 |  updates to the entries that received nonzero gradients in the current
 |  iteration (``mask_updates=True``).
 |  In the context of differentiable Monte Carlo simulations, many of those
 |  variables may not be observed at each iteration, e.g. when a surface is
 |  not visible from the current camera. Gradients for unobserved variables
 |  will remain at zero by default.
 |  If we do not take special care, at each new iteration:
 |  
 |  1. Momentum accumulated at previous iterations (potentially very noisy)
 |   

In [13]:
opt[param_key] = scene_params[param_key]

In [14]:
scene_params.update(opt)

[(SRGBReflectanceSpectrum[
    value = [[0.01, 0, 0.9]]
  ],
  {'value'}),
 (SmoothDiffuse[
    reflectance = SRGBReflectanceSpectrum[
      value = [[0.01, 0, 0.9]]
    ]
  ],
  {'reflectance'}),
 (Scene[
    children = [
      PathIntegrator[
        max_depth = 6,
        rr_depth = 5
      ],
      PerspectiveCamera[
        x_fov = [39.3077],
        near_clip = 10,
        far_clip = 2800,
        film = HDRFilm[
          size = [256, 256],
          crop_size = [256, 256],
          crop_offset = [0, 0],
          sample_border = 0,
          filter = GaussianFilter[stddev=0.50, radius=2.00],
          file_format = OpenEXR,
          pixel_format = rgb,
          component_format = float16,
        ],
        sampler = IndependentSampler[
          base_seed = 0
          sample_count = 1024
          samples_per_wavefront = 1024
          wavefront_size = 67108864
        ],
        resolution = [256, 256],
        shutter_open = 0,
        shutter_open_time = 0,
        to_w

In [15]:
import drjit as dr

def mean_squared_error(image):
    return dr.mean(dr.sqr(image - image_ref))

In [16]:
# gradient decent loop
images = []
for it in range(50):
    # render the scene and compute the loss
    image = mi.render(scene, scene_params, spp=4)
    loss = mean_squared_error(image)

    # trigger jti to backpropogate the loss
    # backpropogate the loss up to the parameter that we enabled the optimizer
    dr.backward(loss)
    opt.step()

    # might need to post process since parameter might left the domain in which
    # is undefined
    opt[param_key] = dr.clamp(opt[param_key], 0.0, 1.0)

    scene_params.update(opt)

    err_ref = dr.sum(dr.sqr(param_ref - scene_params[param_key]))
    print(f"Iteration {it:02d}: parameter error = {err_ref[0]:6f}", end='\r')
    images.append(image)
print('\nOptimization complete.')

Iteration 49: parameter error = 0.001311
Optimization complete.


In [17]:
images

[TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorXf(shape=(256, 256, 3)),
 TensorX

In [19]:
optimized_image = images.pop()

### Optimized Image (Blue wall -> Red Wall)

In [20]:
mi.util.convert_to_bitmap(optimized_image)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]

### Image before Optimization

In [21]:
# initial image
mi.util.convert_to_bitmap(images[0])

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]

### Reference (Original) image

In [22]:
mi.util.convert_to_bitmap(image_ref)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]